In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import logging

from jax import random
import wandb

from src.models import make_Hard_OvR_Ens_loss as make_loss
from src.data import get_image_dataset, NumpyLoader
from src.utils.training import setup_training, train_loop
from experiments.configs.mnist_hard_ovr_classification import get_config

In [ ]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'train_hard_ivr_classifier.ipynb'
# ^ W&B doesn't know how to handle VS Code notebooks.

wandb.login()

In [ ]:
rng = random.PRNGKey(0)

In [ ]:
config = get_config()

In [ ]:
train_dataset, test_dataset, val_dataset = get_image_dataset(
    dataset_name=config.dataset_name,
    val_percent=config.val_percent,
    flatten_img=True,
)
train_loader = NumpyLoader(train_dataset, config.batch_size)
val_loader = NumpyLoader(val_dataset, config.batch_size)
test_loader = NumpyLoader(test_dataset, config.batch_size)

In [ ]:
setup_rng, rng = random.split(rng)
init_x = train_dataset[0][0]
init_y = train_dataset[0][1]

model, state = setup_training(config, setup_rng, init_x, init_y)

In [ ]:
state = train_loop(
    model, state, config, rng, make_loss, make_loss, train_loader, val_loader,
    # test_loader,
    wandb_kwargs={
        'mode': 'offline',
        # 'notes': '',
    },
)